<a href="https://colab.research.google.com/github/RamomNF/GoogleColab/blob/master/TemporadasAnimacao/web_scrapping_animacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Queria por a prova meus conhecimentos de séries temporais, para isso resolvi fazer um web scrapping e pegar
#alguns dados cuja análise não tinha gabarito disponível.
#Análise da Série Temporal: https://github.com/RamomNF/GoogleColab/blob/master/TemporadasAnimacao/series_temporais_animacao.ipynb
#Publicação no Medium: https://medium.com/@ramomnf/evolu%C3%A7%C3%A3o-das-temporadas-de-anime-ao-longo-do-tempo-1c2c3eea205b

In [ ]:
import numpy as np
import pandas as pd
import re
import sys
import time
import urllib.request

from bs4 import BeautifulSoup

In [ ]:
####### Definição de funções

In [ ]:
def get_soup(link):
    try:
        req       = urllib.request.Request(link, headers={'User-Agent': 'Mozilla/5.0'})
        response  = urllib.request.urlopen(req)
        the_page  = response.read()
        soup      = BeautifulSoup(the_page, 'html.parser')

        return soup

    except Exception as ex:
        print("Exception ", ex)
        return None

In [ ]:
def get_ratings(season = '', year =''):
    base_link = 'https://www.anime-planet.com/anime/all?include_types=6&sort=average&order=desc&bvm=list&season='
    page = base_link+season+"-"+str(year)
    print(page)
    names   = list()

    try:
        soup    = get_soup(page)
    except:
        return


    anime = soup.find_all('a',  {'class':'tooltip'})
    
    names = []
    stars = []
    
    for t in anime:
        try:
            stars.append( BeautifulSoup(t['title']).find('div', {'class':'ttRating'}).getText() )        
            names.append( BeautifulSoup(t['title']).find('h5', {'class':'theme-font'}).getText() )
        except:
            pass

    names = names[0:len(stars)]    


    season_df = pd.DataFrame( {'names' : names, 'ratings' : stars} )
    season_df['season'] = season
    season_df['year']   = str(year)

    return season_df

In [ ]:
def get_database(from_year = 1995, to_year = 2021):
    season_list = list()
    for year in range(from_year, to_year):    
        for season in ['winter', 'spring', 'summer', 'fall']:            
            season_list.append( get_ratings(season, str(year)) )

    season_df = pd.concat(season_list)

    return season_df

In [ ]:
####### Execução do webscrapping

In [ ]:
season_df = get_database()

https://www.anime-planet.com/anime/all?include_types=6&sort=average&order=desc&bvm=list&season=spring-1995
https://www.anime-planet.com/anime/all?include_types=6&sort=average&order=desc&bvm=list&season=summer-1995
https://www.anime-planet.com/anime/all?include_types=6&sort=average&order=desc&bvm=list&season=fall-1995
https://www.anime-planet.com/anime/all?include_types=6&sort=average&order=desc&bvm=list&season=winter-1995
https://www.anime-planet.com/anime/all?include_types=6&sort=average&order=desc&bvm=list&season=spring-1996
https://www.anime-planet.com/anime/all?include_types=6&sort=average&order=desc&bvm=list&season=summer-1996
https://www.anime-planet.com/anime/all?include_types=6&sort=average&order=desc&bvm=list&season=fall-1996
https://www.anime-planet.com/anime/all?include_types=6&sort=average&order=desc&bvm=list&season=winter-1996
https://www.anime-planet.com/anime/all?include_types=6&sort=average&order=desc&bvm=list&season=spring-1997
https://www.anime-planet.com/anime/all?in

In [ ]:
season_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2551 entries, 0 to 34
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   names    2551 non-null   object
 1   ratings  2551 non-null   object
 2   season   2551 non-null   object
 3   year     2551 non-null   object
dtypes: object(4)
memory usage: 99.6+ KB


In [ ]:
season_df.describe()

,names,ratings,season,year
count,2551,2551,2551,2551
unique,2550,36,4,26
top,Lalala Lala-chan,3.8,spring,2014
freq,2,161,722,140


In [ ]:
season_df.head()

,names,ratings,season,year
0,Slayers,4.1,spring,1995
1,Tenchi Universe,4,spring,1995
2,H2,3.7,spring,1995
3,Jurassic Tripper,3.6,spring,1995
4,Street Fighter II V,3.4,spring,1995


In [ ]:
season_df.to_csv("data_seasons.csv", sep="\t", index=False)